# Monthly Report ADA Calx
**The below note book is useful to calcuate ADA rewards distributed across various avenues of Indigo Protocol. Indigo Protocol currently has 3 iAsset(iBTC, iETH, iUSD) which have their individual stability pools. These SP pools support the liquidations that occur in the protocol and the depositors of these pools are rewarded with ADA as well as INDY rewards. The values obtained in the below notebook are published as part of the Monthly marketing reports of the protocol and shared with the community across social media platforms.**

In [ ]:
import urllib.request
import pandas as pd
import json
import time
from datetime import datetime
from typing import Final

**The amount of ADA absorbed through liquidations and rewarded to SP stakers in the selected timeframe are as follows:**

In [2]:
# open a connection to a URL using urllib
webUrl  = urllib.request.urlopen('https://analytics.indigoprotocol.io/api/liquidations')

# read the data from the URL and print it
data1 = webUrl.read()
# print (data)

# byteData = b'Lets eat a \xf0\x9f\x8d\x95!'  
b3 = str(data1, 'UTF-8')  
# print(b1)
type(b3)

json_object = json.loads(b3)
# print(type(json_object))

df_ada_sp = pd.DataFrame(json_object)


df_ada_sp=df_ada_sp.drop(['slot','output_hash', 'output_index', 'updated_at'], axis=1)
ch = 'T'

df_ada_sp['created_at'] = df_ada_sp.created_at.str.split(ch).str[0]
df_ada_sp.rename(columns = {'created_at':'date'}, inplace = True)
df_ada_sp
REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379
    
start_day='2024-07-01'
end_day='2024-08-01'

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

def func(value):
    date = pd.to_datetime(value)
    diff = get_difference(date)
    return 379+(diff//5)

df_ada_sp['date']= pd.to_datetime(df_ada_sp['date'])
df_ada_sp
arr_epoch=[]
for i in range(len(df_ada_sp)):
    arr_epoch.append(func(df_ada_sp.loc[i, "date"]))
df_ada_sp.insert(2, column="epoch" , value=arr_epoch)
df_ada_sp['collateral_absorbed'] = df_ada_sp['collateral_absorbed'].astype(float)
df_ada_sp['iasset_burned'] = df_ada_sp['iasset_burned'].astype(float)
df_ada_sp['oracle_price'] = df_ada_sp['oracle_price'].astype(float)
df_ada_sp['ada_price'] = df_ada_sp['ada_price'].astype(float)
df_ada_sp.insert(8, column="value_gained" , value=((df_ada_sp['collateral_absorbed']/1000000)-((df_ada_sp['iasset_burned']/1000000)*df_ada_sp['oracle_price']))*df_ada_sp['ada_price'])
# df_ada_sp
new2= df_ada_sp.loc[(df_ada_sp['date'] >= start_day)
                         & (df_ada_sp['date'] <= end_day)]
columns_to_drop = ['id', 'iasset_burned','oracle_price','ada_price', 'value_gained','date','epoch']
new2 = new2.drop(columns_to_drop, axis=1)
rew_buffer1=new2.groupby('asset')
rew_buffer1.sum()/1000000

,collateral_absorbed
asset,
iETH,2359.220248
iUSD,41718.058759


In [4]:
import urllib.request
import pandas as pd
import json
import time
from datetime import datetime
from typing import Final

# open a connection to a URL using urllib
webUrl  = urllib.request.urlopen('https://analytics.indigoprotocol.io/api/distributions')

# read the data from the URL and print it
data = webUrl.read()
# print (data)

# byteData = b'Lets eat a \xf0\x9f\x8d\x95!'  
b1 = str(data, 'UTF-8')  
# print(b1)
type(b1)

json_object = json.loads(b1)
# print(type(json_object))

df = pd.DataFrame(json_object)

REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

def func(value):
    full_date = time.strftime("%d %b %Y", time.gmtime(value))
    date = pd.to_datetime(full_date)
    diff = get_difference(date)
    return 379+(diff//5)
    
arr=[]
for i in range(len(df)):
    arr.append(func(df.loc[i, "timestamp"]))
df.insert(2, column="epoch" , value=arr)
df.insert(3, column="actual_ada_distributed", value=df['ada_distributed']/1000000)

df['indy_ada_price'] = df['indy_ada_price'].astype(float)
df.insert(8, column="actual_value_in_INDY", value=df['actual_ada_distributed']/df['indy_ada_price'])
columns_to_drop = ['indy_usd_price','ada_usd_price','timestamp','slot']
df = df.drop(columns=columns_to_drop)

dk2=df.groupby('epoch')
dk22=dk2.sum()
dk22


,ada_distributed,actual_ada_distributed,indy_ada_price,actual_value_in_INDY
epoch,,,,
379,2529227046,2529.227046,59.291120,256.056506
380,15535063962,15535.063962,272.182234,1498.551186
381,3233052393,3233.052393,211.652067,314.123092
382,8037186950,8037.186950,287.352916,934.124760
383,7054932897,7054.932897,356.839412,888.959437
...,...,...,...,...
509,463804227,463.804227,9.723886,334.642973
510,15769303161,15769.303161,14.500850,11415.118762
511,188909883,188.909883,4.639319,162.866187


**The total amount of ADA distributed to INDY stakers in the selected timeframe is as follows:**

In [5]:

dk22.index.name = 'epoch'
selected_rows = dk22.loc[[502,503,504,505,506,507]]
print(selected_rows.to_string())
print("Total ADA distributed:"+ str(selected_rows['actual_ada_distributed'].sum()))


       ada_distributed  actual_ada_distributed  indy_ada_price  actual_value_in_INDY
epoch                                                                               
502        24903007327            24903.007327       65.082879          15122.054120
503         1093906331             1093.906331       22.117574            753.299030
504        15877932111            15877.932111       11.556069          11133.793116
505          953874568              953.874568       18.259441            629.702321
506         1949910174             1949.910174       30.520843           1274.561287
507        15999384042            15999.384042       15.931183          11050.742643
Total ADA distributed:60778.01455299999
